# PCA and Clustering Visualization
In this notebook, we will apply Principal Component Analysis (PCA) to reduce the dimensionality of the dataset. Then, we will visualize the clusters obtained from different clustering methods.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

## Load the Data
First, let's load the data that contains the clustering results.

In [ ]:
# Load the data
hado_data = pd.read_csv('hado_data_clustering.csv')

# Show the first few rows of the DataFrame
hado_data.head()

In [ ]:
hado_data = hado_data.rename(columns={'diagnostico_cluster_optimal': 'diagnostico_kmeans',
                                      'motivo_ing_cluster_optimal': 'motivo_ing_kmeans',
                                      'motivo_alta_cluster_optimal': 'motivo_alta_kmeans',
                                      'diagnostico_topic': 'diagnostico_lda'
                                     }
                            )

## Apply PCA
Now, let's apply Principal Component Analysis (PCA) to reduce the dimensionality of the TF-IDF features. We will reduce it to 2 dimensions for easy visualization.

In [ ]:
hado_data = hado_data.dropna(subset=['diagnostico'])
hado_data.reset_index(drop=True, inplace=True)

In [ ]:
# Importar la biblioteca necesaria para TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear un objeto TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Ajustar y transformar los datos de la columna 'diagnostico' al formato TF-IDF
diagnostico_tfidf = tfidf_vectorizer.fit_transform(hado_data['diagnostico'])

In [ ]:
# Initialize PCA and the X vector for dimensionality reduction
pca = PCA(n_components=2)

# Assuming that the TF-IDF features are stored in a variable named 'diagnostico_tfidf'
X = diagnostico_tfidf.toarray()

# Apply PCA
principalComponents = pca.fit_transform(X)

# Create a DataFrame with the principal components
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

# Show the first few rows of the DataFrame
principalDf.head()

## Visualize Clusters
Now that we have reduced the dimensionality of our data, let's visualize the clusters obtained from different clustering methods.

In [ ]:
# Merge the principal components DataFrame with the original DataFrame
finalDf = pd.concat([principalDf, hado_data[['diagnostico_kmeans', 'diagnostico_lda', 'diagnostico_hierarchical', 'diagnostico_dbscan']]], axis = 1)

# Plotting the clusters obtained from different methods
fig, axes = plt.subplots(4, 1, figsize=(8, 20))

for ax, method in zip(axes, ['diagnostico_kmeans', 'diagnostico_lda', 'diagnostico_hierarchical', 'diagnostico_dbscan']):
    sns.scatterplot(x='principal component 1', y='principal component 2', hue=method, data=finalDf, ax=ax)
    ax.set_title(f'Clusters from {method}')

plt.show()

### Interpretación y Conclusiones

#### Observaciones Clave
1. **PCA**: La reducción de dimensionalidad a través de PCA parece efectiva para visualizar los datos en un espacio bidimensional.
  
2. **Visualización de Clusters**: 
    - **K-means**: Los clusters parecen bastante separados, lo que indica una buena agrupación.
    - **LDA**: Los clusters están menos separados en comparación con K-means.
    - **Clustering Jerárquico**: Similar a K-means, los clusters parecen bien separados.
    - **DBSCAN**: Este método parece identificar un cluster muy grande y varios más pequeños, lo que podría indicar la presencia de ruido o outliers.

3. **Métricas de Validación**:
    - **Validación Interna**: DBSCAN tiene el puntaje de Silhouette más alto, lo que generalmente indica una mejor separación de los clusters. Sin embargo, este método también tiene el puntaje de Davies-Bouldin más bajo, lo que sugiere que los clusters no están tan bien definidos.
    - **Validación Externa**: El Clustering Jerárquico tiene el índice Rand ajustado más alto y la información mutua normalizada más alta, lo que sugiere que este método es el más preciso en términos de asignación de clusters en comparación con las etiquetas verdaderas.

#### Conclusiones
- El PCA es efectivo para reducir la dimensionalidad de los datos y facilitar la visualización.
- K-means y el Clustering Jerárquico parecen ser los métodos más efectivos para este conjunto de datos, según las métricas de validación.
- DBSCAN podría requerir más ajustes para mejorar la calidad de los clusters.
- La validación externa sugiere que el Clustering Jerárquico es el más alineado con las etiquetas verdaderas, lo que podría hacerlo más útil para interpretaciones clínicas.

#### Pasos Futuros
- Podría ser útil probar otros métodos de clustering o ajustar los parámetros de los métodos actuales para mejorar la calidad de los clusters.
- También sería interesante explorar cómo las diferentes características (como las variables clínicas) se distribuyen entre los clusters para obtener más información clínica.